In [3]:
import pycosat
import numpy as np

# Global sudoku length
N = 9

In [2]:
# Link to encoding a cell: https://nickp.svbtle.com/sudoku-satsolver, https://github.com/ContinuumIO/pycosat/blob/master/examples/sudoku.py
def transform(cell_row, cell_col, digit):
    return int(cell_row*N*N + cell_col*N + digit)

def ind_cell_encoding(encodings):
    for i in range(N):
        for j in range(N):
            all_possible_values = []
            for d in range(1, 10):
                d_transform = transform(i, j, d)
                all_possible_values.append(d_transform)
                for other_d in range(d+1, 10):
                    encodings.append([-d_transform, -transform(i, j, other_d)])
            encodings.append(all_possible_values)
    return encodings

def row_cell_encoding(encodings):
    for i in range(N):
        for d in range(1, 10):
            all_possible_values = []
            for j in range(N):
                d_transform = transform(i, j, d)
                all_possible_values.append(d_transform)
                for other_j in range(j+1, N):
                    encodings.append([-d_transform, -transform(i, other_j, d)])
            encodings.append(all_possible_values)
    return encodings
    
def col_cell_encoding(encodings):
    for j in range(N):
        for d in range(1, 10):
            all_possible_values = []
            for i in range(N):
                d_transform = transform(i, j, d)
                all_possible_values.append(d_transform)
                for other_i in range(i+1, N):
                    encodings.append([-d_transform, -transform(other_i, j, d)])
            encodings.append(all_possible_values)
    return encodings
    
     
def block_cell_encoding(encodings):
    for i in range(0, N, 3):
        for j in range(0, N, 3):
            for d in range(1, 10):
                all_possible_values = []
                for k in range(3):
                    for l in range(3):
                        d_transform = transform(i+k, j+l, d)
                        all_possible_values.append(d_transform)
                        for other_d in range(d+1, 10):
                            encodings.append([-d_transform, -transform(i+k, j+l, other_d)])  
                encodings.append(all_possible_values)
    return encodings
                
def filled_in_encoding(encodings, sudoku):
    for i in range(N):
        for j in range(N):
            current_number = sudoku[i][j]
            if current_number > 0:
                encodings.append([transform(i, j, current_number)])
    return encodings
    
def encoding(sudoku):
    encodings = []
    encodings = ind_cell_encoding(encodings)
    encodings = row_cell_encoding(encodings)
    encodings = col_cell_encoding(encodings)
    encodings = block_cell_encoding(encodings)
    encodings = filled_in_encoding(encodings, sudoku)
    return encodings

def reverse_encoding(solved):
    if isinstance(solved, str):
        return "Sudoku could not be solved: %s" % solved
    sudoku = np.zeros((N, N))
    for number in solved:
        if number > 0:
            number, d = divmod(number, N)
            if d == 0:
                number = number - 1
                d = 9
            number, cell_col = divmod(number, N)
            number, cell_row = divmod(number, N)
            sudoku[cell_row][cell_col] = d
    return sudoku

def solver(sudoku):
    encodings = encoding(sudoku)
    print("Encoding length naive: {0} clauses".format(len(encodings)))
    solved = pycosat.solve(encodings)
    return(reverse_encoding(solved))

In [ ]:
sudoku = np.load("data\Easy\sudoku-Easy-0.npy")
print("Sudoku to solve:")
print(sudoku)
print("Solved sudoku:")
solved_sudoku = solver(sudoku)
print(solved_sudoku)

In [1]:
def encoding_efficient(sudoku):
    encodings = []
    encodings, filled_in = filled_in_encoding_efficient(encodings, sudoku)
    encodings = ind_cell_encoding_efficient(encodings, filled_in)
    encodings = row_cell_encoding_efficient(encodings, filled_in)
    encodings = col_cell_encoding_efficient(encodings, filled_in)
    encodings = block_cell_encoding_efficient(encodings, filled_in)
    return encodings

def filled_in_encoding_efficient(encodings, sudoku):
    filled_in = []
    for i in range(N):
        for j in range(N):
            current_number = sudoku[i][j]
            if current_number != 0:
                encoded_number = transform(i, j, current_number)
                encodings.append([encoded_number])
                filled_in.append(encoded_number)
    return encodings, filled_in

def ind_cell_encoding_efficient(encodings, filled_in):
    for i in range(N):
        for j in range(N):
            all_possible_values = []
            exactly_one = []
            for d in range(1, 10):
                d_transform = transform(i, j, d)
                if d_transform not in filled_in:
                    all_possible_values.append(d_transform)
                    for other_d in range(d+1, 10):
                        otherd_transform = transform(i, j, other_d)
                        if otherd_transform not in filled_in:  
                            encodings.append([-d_transform, -otherd_transform])
            encodings.append(all_possible_values)
    return encodings

def row_cell_encoding_efficient(encodings, filled_in):
    for i in range(N):
        for d in range(1, 10):
            all_possible_values = []
            exactly_one = []
            for j in range(N):
                d_transform = transform(i, j, d)
                if d_transform not in filled_in:                
                    all_possible_values.append(d_transform)
                    for other_j in range(j+1, N):
                        otherj_transform = transform(i, other_j, d)
                        if otherj_transform not in filled_in:
                            encodings.append([-d_transform, -otherj_transform])
            encodings.append(all_possible_values)
    return encodings
    
def col_cell_encoding_efficient(encodings, filled_in):
    for j in range(N):
        for d in range(1, 10):
            all_possible_values = []
            exactly_one = []
            for i in range(N):
                d_transform = transform(i, j, d)
                if d_transform not in filled_in:                
                    all_possible_values.append(d_transform)
                    for other_i in range(i+1, N):
                        otheri_transform = transform(other_i, j, d)
                        if otheri_transform not in filled_in:
                            encodings.append([-d_transform, -otheri_transform])
            encodings.append(all_possible_values)
    return encodings
    
     
def block_cell_encoding_efficient(encodings, filled_in):
    for i in range(0, N, 3):
        for j in range(0, N, 3):
            for d in range(1, 10):
                all_possible_values = []
                exactly_one = []
                for k in range(0, 3):
                    for l in range(0, 3):
                        d_transform = transform(i+k, j+l, d)
                        if d_transform not in filled_in:
                            all_possible_values.append(d_transform)
                            for other_d in range(d+1, 10):
                                otherd_transform = transform(i+k, j+l, other_d)
                                if otherd_transform not in filled_in:
                                    encodings.append([-d_transform, -otherd_transform])  
                encodings.append(all_possible_values)
    return encodings

def solver_efficient(sudoku):
    encodings = encoding_efficient(sudoku)
    print("Encoding length efficient: {0} clauses".format(len(encodings)))
    solved = pycosat.solve(encodings)
    return reverse_encoding(solved)



In [4]:
sudoku = np.load("data\Easy\sudoku-Easy-0.npy")
print(sudoku)
solved_sud = solver_efficient(sudoku)
print(solved_sud)

[[0 0 0 2 1 6 0 7 4]
 [2 0 0 0 3 0 0 5 6]
 [0 0 0 5 8 9 0 3 2]
 [1 9 0 4 0 0 0 0 7]
 [0 0 7 1 9 2 0 0 0]
 [0 2 4 6 7 0 0 0 9]
 [0 0 0 0 0 0 0 0 0]
 [5 1 2 9 0 0 0 4 0]
 [7 4 0 8 2 1 0 0 0]]
Encoding length efficient: 10841 clauses
[[ 9.  8.  7.  6.  5.  6.  3.  7.  4.]
 [ 6.  3.  2.  1.  9.  7.  5.  5.  8.]
 [ 5.  4.  1.  5.  8.  9.  6.  7.  9.]
 [ 8.  9.  9.  5.  6.  3.  4.  1.  7.]
 [ 4.  6.  7.  2.  9.  2.  9.  3.  7.]
 [ 2.  2.  4.  7.  7.  9.  8.  6.  9.]
 [ 3.  5.  6.  9.  1.  2.  7.  8.  4.]
 [ 7.  2.  4.  9.  3.  5.  1.  9.  6.]
 [ 7.  9.  8.  8.  7.  6.  2.  5.  3.]]
